In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# from pprint import pprint
# from lxml import html
# from lxml.html import fromstring
#import urllib.request
#from urllib.request import urlopen
#from selenium import webdriver
# import random
#import re
# import scrapy
#

In [70]:
url = 'https://www.archives.gov/electoral-college/1980'
html = requests.get(url).content
table = pd.read_html(html)
#soup = BeautifulSoup(html, "lxml")
#soup

In [72]:
results_df = table[1]

In [74]:
results_df.columns = ['state','electoral_votes'] + list(results_df.iloc[0,0:4])

In [75]:
results_df.insert(0,'year',1980)

In [76]:
# drop header row
results_df = results_df.drop(results_df.index[0])

In [78]:
# drop VP Columns
num_veeps = int((results_df.shape[1]-3)/2)
veep_names = results_df.columns[-num_veeps:]
results_df = results_df.drop(veep_names, axis = 1)

In [85]:
results_df.melt(id_vars =['year', 'state'], value_vars = results_df.columns[3:], var_name = 'candidate',value_name='electoral_votes')

,year,state,candidate,electoral_votes
0,1980,Alabama,"Ronald Reagan, of California",9
1,1980,Alaska,"Ronald Reagan, of California",3
2,1980,Arizona,"Ronald Reagan, of California",6
3,1980,Arkansas,"Ronald Reagan, of California",6
4,1980,California,"Ronald Reagan, of California",45
...,...,...,...,...
99,1980,Washington,"Jimmy Carter, of Georgia",-
100,1980,West Virginia,"Jimmy Carter, of Georgia",6
101,1980,Wisconsin,"Jimmy Carter, of Georgia",-
102,1980,Wyoming,"Jimmy Carter, of Georgia",-


In [91]:
df = pd.DataFrame()
for year in range(1976,2020,4):
    url = f'https://www.archives.gov/electoral-college/{year}'
    html = requests.get(url).content
    table = pd.read_html(html)
    results_df = table[1]
    results_df.columns = ['state','electoral_votes'] + list(results_df.iloc[0,0:-2])
    results_df.insert(0,'year', year)
    
    # drop header row
    results_df = results_df.drop(results_df.index[0])
    
    # drop VP Columns
    num_veeps = int((results_df.shape[1]-3)/2)
    veep_names = results_df.columns[-num_veeps:]
    results_df.drop(veep_names, axis = 1, inplace = True)
    
    df = df.append(results_df.melt(id_vars =['year', 'state'], value_vars = results_df.columns[3:], var_name = 'candidate',value_name='electoral_votes'))

In [99]:
df['electoral_votes'].mask(df['electoral_votes'] == "-",0, inplace =  True)

In [100]:
df[df['year'] == 2012]

,year,state,candidate,electoral_votes
0,2012,Alabama,"Barack Obama, of Illinois",0
1,2012,Alaska,"Barack Obama, of Illinois",0
2,2012,Arizona,"Barack Obama, of Illinois",0
3,2012,Arkansas,"Barack Obama, of Illinois",0
4,2012,California,"Barack Obama, of Illinois",55
...,...,...,...,...
99,2012,Washington,"Mitt Romney, of Massachusetts",0
100,2012,West Virginia,"Mitt Romney, of Massachusetts",5
101,2012,Wisconsin,"Mitt Romney, of Massachusetts",0
102,2012,Wyoming,"Mitt Romney, of Massachusetts",3


In [101]:
df.to_csv("data/electoral_college.csv", index= False)